In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import imageio
import os
from glob import glob
import nibabel as nib
from tqdm import tqdm
import json
import nibabel as nib
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import nibabel as nib
from glob import glob
from tqdm import tqdm
import pandas as pd
import scipy.ndimage
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [6]:
def center_crop_or_pad_2d(volume, target_hw=(384, 384), pad_value=-1):
    """
    volume: (D, H, W)
    target_hw: (target_height, target_width)
    pad_value: value to fill if padding is needed
    """
    d, h, w = volume.shape
    target_h, target_w = target_hw

    padded = np.full((d, target_h, target_w), fill_value=pad_value, dtype=volume.dtype)

    crop_h = min(h, target_h)
    crop_w = min(w, target_w)

    start_h = (h - crop_h) // 2
    start_w = (w - crop_w) // 2

    target_start_h = (target_h - crop_h) // 2
    target_start_w = (target_w - crop_w) // 2

    padded[:, target_start_h:target_start_h + crop_h,
              target_start_w:target_start_w + crop_w] = volume[:, start_h:start_h + crop_h,
                                                               start_w:start_w + crop_w]
    return padded

def resize_z(volume, target_depth=32):
    """volume: (D, H, W)"""
    current_depth = volume.shape[0]
    zoom_factor = target_depth / current_depth
    volume_resized = scipy.ndimage.zoom(volume, zoom=(zoom_factor, 1, 1), order=0)
    return volume_resized

def load_and_preprocess_nifti(path, target_shape=(32, 384, 384)):
    # Load NIfTI
    nifti = nib.load(path)
    volume = nifti.get_fdata()

    # Z 방향 resize
    volume = resize_z(volume, target_depth=target_shape[0])

    # X, Y 방향 crop 또는 padding
    volume = center_crop_or_pad_2d(volume, target_hw=(target_shape[1], target_shape[2]), pad_value=-1)

    return volume.astype(np.float32)

# 경로 설정
data_path = '../../data/ct_nii/'
save_dir = '../../data/prepross_img/ct/'
data_list = glob(data_path + '*.nii.gz')
save_list = [f.replace('ct_nii/', 'prepross_img/ct/') for f in data_list]

os.makedirs(save_dir, exist_ok=True)

for i in tqdm(range(len(data_list))):
    vol = load_and_preprocess_nifti(data_list[i])  # (32, 384, 384)
    nifti_img = nib.Nifti1Image(vol, affine=np.eye(4))
    nib.save(nifti_img, save_list[i])

  0%|          | 0/809 [00:00<?, ?it/s]

100%|██████████| 809/809 [10:50<00:00,  1.24it/s]


In [14]:
nifti = nib.load(data_list[0])
volume = nifti.get_fdata()
volume.shape

(23, 288, 384)